In [7]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

In [8]:
def imprimeResult(results):
    media = results['test_score'].mean()
    print("Resultados cross validation:",results['test_score'])
    print("Média da precisão: {0:.2f}".format(media*100),'%')

    #Acurracy fica entre a média + 2(desvio_padrao) e a média - 2(desvio_padrao)
    desvio_padrao = results['test_score'].std()
    print("Desvio padrão: {0:.2f}".format(desvio_padrao*100),'%')

    acumulo_baixo = media - 2*desvio_padrao
    acumulo_alto = media + 2*desvio_padrao
    print("Acurracy esta entre: [",acumulo_baixo*100,"---", acumulo_alto*100,"]")

<h2>Criando uma nova coluna (aleatória)</h2>
<p>
   A coluna representa a possibilidade de em um futuro o modelo ter que prever com dados não treinados previamente 
</p>

In [9]:
import numpy as np

SEED = 123

np.random.seed(SEED)
np.random.randint(-2,2, size = 10000) #size é o numero de linhas do dataFrame

dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2,2, size = 10000)
#Deixa o menor valor da coluna possitivo
abss = abs(dados['modelo_aleatorio'].unique().min())
dados['modelo_aleatorio'] = dados['modelo_aleatorio'] + abss + 1    #somando 1 para que não seja 0

#Aleatoriedade adicionada e, alguns valores aparecem mais que outros -> era isso que queriamos!
dados['modelo_aleatorio'].value_counts()

20    937
19    850
18    788
21    685
15    680
16    677
17    671
14    619
13    590
12    506
11    506
22    487
10    373
9     313
8     296
23    256
7     245
6     192
5     142
4      93
3      61
2      27
1       6
Name: modelo_aleatorio, dtype: int64

<h1> Pipeline</h1>
<p>
    O pipeline é um objeto que é responsável por executar todas as etapas do processo de previsão. <br>
    Para utilizarmos o SVC() é necessário utilizar um StandardScaler() para normalizar os dados. <br>
    Porém sem o pipeline, seria necessário fazer o mesmo processo para cada etapa (cada fold) do cross validation. <br>
    Com o pipeline, é possível automatizar esse processo. <br>
</p>

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
model = SVC()

pipeline = Pipeline([('scaler', scaler), ('model', model)])

cv = GroupKFold(n_splits=5)

#AO invés do modelo, aqui é passado o pipeline
result = cross_validate(pipeline, x, y , cv=cv, groups = dados['modelo_aleatorio'])

imprimeResult(result)

Resultados cross validation: [0.77042607 0.77527528 0.74887444 0.7703373  0.77158635]
Média da precisão: 76.73 %
Desvio padrão: 0.94 %
Acurracy esta entre: [ 74.85280630524154 --- 78.60717067978325 ]
